# Lab 2. Search
# Task 2.3 Maze Solver
## Problem Descriptions
The maze solver aims to find the shortest path from the starting point ('0') to the target point ('X') in a maze through the permissible paths and avoiding walls. The maze is constructed by walls ('#') and open path (blank space). The search problem can be defined as follows.

1. State: Current position within the maze and is represented by (x,y)
  coordinates.

2. Action: Valid horizontal and vertical movement can be made in the maze to reach the target point ('X') while avoiding the wall ('#'), which includes up, down, left, and right. For an extended search, the valid movement is expanded to move diagonally, which includes up left, up right, down left, and down right.

3. Goal test： Check whether the current state is the target point ('X').

4. Path cost： Total movement used to reach the target point ('X) from the starting point ('0'). Each vertical or horizontal movement costs 1 and each diagonal movement costs 1.4.

5. Heuristic function: Euclidean distance, is the straight line distance from starting point to the target point. It is suitable especially when diagonal movements are allowed. It can be calculated by using the Pythagorean theorem.
                      
                            c=sqrt(a^2+b^2)


## Implementation and Results

In [ ]:
!pip install simpleai
from simpleai.search import SearchProblem, astar, greedy, breadth_first, depth_first, uniform_cost
import math

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.4/94.4 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for simpleai: filename=simpleai-0.8.3-py3-none-any.whl size=100984 sha256=446d9572c95db58ec70985ea31fc8ed79f1e48004790f63d46356b2daf1aaa16
  Stored in directory: /root/.cache/pip/wheels/91/0c/38/421d7910e7bc59b97fc54f490808bdb1097607d83d1a592865
Successfully built simpleai


In [ ]:
MAP = """
##############################
#         #              #   #
# ####    ########       #   #
#  o #    #              #   #
#    ###     ####   ######   #
#         ####      #        #
#            #  #   #   #### #
#     ######    #       # x  #
#        #      #            #
##############################
"""
MAP = [list(x) for x in MAP.split("\n") if x]

COSTS = {
    "up": 1.0,
    "down": 1.0,
    "left": 1.0,
    "right": 1.0,
    "up left": 1.4,
    "up right": 1.4,
    "down left": 1.4,
    "down right": 1.4,
}


In [ ]:

class Maze(SearchProblem):

    def __init__(self, board):
        self.board = board
        self.goal = (0, 0)
        for y in range(len(self.board)):
            for x in range(len(self.board[y])):
                if self.board[y][x].lower() == "o":
                    self.initial = (x, y)
                elif self.board[y][x].lower() == "x":
                    self.goal = (x, y)

        super(Maze, self).__init__(initial_state=self.initial)

    def actions(self, state):
        actions = []
        for action in list(COSTS.keys()):
            newx, newy = self.result(state, action)
            if self.board[newy][newx] != "#":
                actions.append(action)
        return actions

    def result(self, state, action):
        x, y = state

        if action.count("up"):
            y -= 1
        if action.count("down"):
            y += 1
        if action.count("left"):
            x -= 1
        if action.count("right"):
            x += 1

        new_state = (x, y)
        return new_state

    def is_goal(self, state):
        return state == self.goal

    def cost(self, state, action, state2):
        return COSTS[action]

    def heuristic(self, state):
        x, y = state
        gx, gy = self.goal
        return math.sqrt((x - gx) ** 2 + (y - gy) ** 2)

In [ ]:
problem = Maze(MAP)
#result = astar(problem, graph_search=True)
#result = greedy(problem, graph_search=True)
#result = breadth_first(problem, graph_search=True)
#result = depth_first(problem, graph_search=True)
#result = uniform_cost(problem, graph_search=True)

path = [x[1] for x in result.path()]
for y in range(len(MAP)):
    for x in range(len(MAP[y])):
        print('.' if (x,y) in path[1:-1] else MAP[y][x], end='')
    print()
print("A* Search: moves=%s, cost=%s." %(result.depth, result.cost))

##############################
#         #              #   #
# ####    ########       #   #
#  o #    #  ....        #   #
#   .###  ...####.  ######   #
#    .....####    . #        #
#            #  #  .#   #### #
#     ######    #   ....# x  #
#        #      #       ..   #
##############################
A* Search: moves=23, cost=26.999999999999993.


## Discussions
Using initial search

In the initial search, we can only perform horizontal and vertical movements while avoiding the wall. After applying all search algorithms, all A* search, greedy best-first search, breadth-first search, depth-first search, and uniform cost search use 33 moves with 33 path costs to reach the target point. Such a situation might be due to limited movement options available, there are no alternative paths to reach the target if only using horizontal and vertical movements.

Using extended search

In extended search, we can perform horizontal, vertical, and diagonal movements while avoiding the wall. The movement and path costs for each search algorithm are collected as follows.


            Search algorithm   no.of movement         path cost
                   A* search               23             26.99
    greedy best-first search               23             26.99    
        breadth-first search               23             26.99
          depth-first search               38             49.59
         uniform cost search               23             26.99


From the result, the introduction of diagonal movements has significantly reduced the path cost required by A* search, breadth-first search, and uniform cost search from 33 to 26.99. However, the depth-first search took a very long path to reach the target, which might be due to increased movement complexity. Depth-first search tends to explore the deepest end of the movement option made and is likely to get trapped into longer alternative paths.

It is not impossible that A* search, breadth-first search, and uniform cost search still give an identical path cost again given more complex movement options. All A* search, uniform cost search, and greedy best-first search consider the path cost during path exploration, but in different ways. Although the breadth-first search didn't take path costs into account, there is a possibility that it finds an optimal path when exploring all the shallowest paths before going to deep.

In conclusion, searching path cost can be minimized given more complex movement options but it might not work on depth-first search. Depth-first search is likely to get trapped into longer alternative paths due to its searching nature.

